# Lab 6 Iterativer Phasenschätzungsalgorithmus

Voraussetzung

- [Kap.3.5 Quanten-Fourier-Transformation](https://qiskit.org/textbook/ch-algorithms/quantum-fourier-transform.html)
- [Kap.3.6 Quantenphasenschätzung](https://qiskit.org/textbook/ch-algorithms/quantum-phase-estimation.html)
- [Kapitel 1.4 Einzelne Qubit-Gatter](https://qiskit.org/textbook/ch-states/single-qubit-gates.html)
- [Zusammenfassung der Quantenoperationen](https://qiskit.org/documentation/tutorials/circuits/3_summary_of_quantum_operations.html)

Andere relevante Materialien

- [Geräte-Backend-Rauschmodellsimulationen](https://qiskit.org/documentation/tutorials/simulators/2_device_noise_simulation.html)
- [Hellinger Treue](https://qiskit.org/documentation/stubs/qiskit.quantum_info.hellinger_fidelity.html#qiskit.quantum_info.hellinger_fidelity)

In [13]:
from qiskit import *
import numpy as np
from qiskit.visualization import plot_histogram
import qiskit.tools.jupyter
import matplotlib.pyplot as plt

In [14]:
sim = Aer.get_backend('qasm_simulator')

<h2 style="font-size:24px;">Teil 1: Implementierung des iterativen Phasenschätzungsalgorithmus</h2>

<br>
<div style="background: #E8E7EB; border-radius: 5px;
-moz-border-radius: 5px;">
  <p style="background: #800080;
            border-radius: 5px 5px 0px 0px;
            padding: 10px 0px 10px 10px;
            font-size:18px;
            color:white;
            "><b>Tor</b></p>
    <p style=" padding: 0px 0px 10px 10px;
              font-size:16px;">Schätzen Sie einen Phasenwert auf einem System aus zwei Qubits durch den Iterative Phase Estimation (IPE)-Algorithmus.</p>
</div>

Nachdem Sie frühere Übungen durchlaufen haben, sollten Sie bemerkt haben, dass die "Länge" eines Quantenschaltkreises der Hauptfaktor bei der Bestimmung der Größe der Fehler in der resultierenden Ausgabeverteilung ist; Quantenschaltkreise mit größerer Tiefe haben eine verringerte Wiedergabetreue. Daher ist die Implementierung von Algorithmen, die auf Schaltungen mit geringer Tiefe basieren, von großer Bedeutung für das kurzfristige Quantencomputing. In Lab 5 lernen wir einen solchen Algorithmus zur Schätzung der Quantenphase kennen, den Iterative Phase Estimation (IPE)-Algorithmus, der ein System erfordert, das nur aus einem einzigen Hilfs-Qubit besteht, und die Phase durch einen sich wiederholenden Prozess auswertet. 

<h3 style="font-size: 20px">1. Verstehen Sie eine Schaltung mit nicht unitären Operationen.</h3>

Bevor wir lernen, wie der IPE-Algorithmus funktioniert, lassen Sie uns die Reset- und Bedingungsoperationen in Qiskit betrachten, die zum Aufbau einer IPE-Schaltung beitragen. Lesen Sie [hier](https://qiskit.org/documentation/tutorials/circuits/3_summary_of_quantum_operations.html) das Qiskit-Tutorial ( gehen Sie zum Abschnitt `Non-unitary operations` ), um zu verstehen, wie man eine Schaltung baut, die bedingte Operationen ausführt und zurücksetzt.

<h4 style="font-size: 17px">📓Schritt A. Führen Sie die folgende Zelle aus und sagen Sie das Ergebnis der Schaltung voraus.</h4>

In [2]:
q = QuantumRegister(1)
c = ClassicalRegister(2)

qc0 = QuantumCircuit(q, c)
qc0.h(q[0])
qc0.measure(q[0], c[0])
qc0.reset(q[0])
qc0.h(q[0])
qc0.p(np.pi/3, q[0]).c_if(c,1)
qc0.h(q[0])
qc0.measure(q[0],c[1])

qc0.draw()

Führen Sie die Zelle unten aus, um das Ergebnis anzuzeigen.

In [3]:
count0 = execute(qc0, sim).result().get_counts()
plot_histogram(count0)

<h4 style="font-size: 17px">📓Schritt B. Vervollständige den Rest der Schaltung, sodass das Hilfs-Qubit ( oberstes Qubit ) nach dem Zurücksetzen im Zustand $\frac{1}{\sqrt2}(|0\rangle + e^{-i\frac {\pi}{2}}|1\rangle)$ wenn der Wert des klassischen Bits Eins ist oder ansonsten Nullzustand bleibt.</h4>

In [1]:
q = QuantumRegister(2)
c = ClassicalRegister(2)

qc1 = QuantumCircuit(q,c)
qc1.h(q[0])
qc1.x(q[1])
qc1.cp(np.pi/5, q[0], q[1])
qc1.measure(q[0], c[0])
qc1.reset(q[0])

###### your code goes here #####





##########################
qc1.h(q[0])
qc1.measure(q[0],c[1])

qc1.draw()

Führen Sie die folgende Zelle aus, um das Ergebnis anzuzeigen.

In [4]:
count1 = execute(qc1, sim).result().get_counts()
plot_histogram(count1)

<h3 style="font-size: 20px">2. Algorithmus zur iterativen Phasenschätzung (IPE).</h3>

Die Quantum Phase Estimation (QPE)-Schaltung, die wir gelernt und zuvor verwendet haben, ist durch die Anzahl der Qubits begrenzt, die für die Genauigkeit des Algorithmus erforderlich sind. Jedes zusätzliche Qubit hat zusätzliche Kosten in Bezug auf Rauschen und Hardwareanforderungen; verrauschte Ergebnisse, die wir bei der Ausführung der QPE-Schaltung auf einem echten Quantengerät in Lab 4 erhalten haben, würden sich mit zunehmender Anzahl der Qubits auf der Schaltung verschlimmern. Der IPE-Algorithmus implementiert die Quantenphasenschätzung mit nur einem einzigen Hilfs-Qubit, und die Genauigkeit des Algorithmus ist eher durch die Anzahl der Iterationen als durch die Anzahl der zählenden Qubits beschränkt. Daher sind IPE-Schaltungen von praktischem Interesse und von größter Bedeutung für das kurzfristige Quantencomputing, da QPE ein wesentliches Element in vielen Quantenalgorithmen ist.

Betrachten Sie das Problem, $\varphi$ zu finden, wenn $|\Psi\rangle$ und $U$ gegeben sind, so dass $U |\Psi\rangle = e^{i \phi} | \Psi \rangle$, mit $\phi = 2 \pi \varphi$. Nehmen wir zunächst an, dass $\varphi$ geschrieben werden kann als $\varphi = \varphi_1/2 + \varphi_2/4 + ... + \varphi_m/2^m = 0.\varphi_1 \varphi_2 ... \varphi_m$ , wobei wir zuvor die Notation $0.\varphi_1 \varphi_2 ... \varphi_m$ definiert haben.

Angenommen, $U$ ist ein unitärer Operator, der auf ein Qubit wirkt. Wir brauchen daher ein System aus zwei Qubits, $q_0$ und $q_1$, wobei $q_0$ ein Hilfs-Qubit ist und das Qubit $q_1$ das physische System darstellt, auf dem $U$ arbeitet. Wurden sie als $q_0 \rightarrow |+\rangle$ und $q_1 \rightarrow |\Psi \rangle$ initialisiert, würde die Anwendung von Control-U zwischen $q_0$ und $q_1$ $2^t$ Mal den Zustand von $q_0 ändern $ zu $|0\rangle + e^{i 2 \pi 2^{t} \varphi} | 1 \rangle$. Das heißt, die Phase von $U$ wurde so oft in $q_0$ zurückgesetzt, wie die Steueroperation durchgeführt wurde.

Deshalb,

für $t=0$ wäre die in $q_0$ codierte Phase $e^{i 2 \pi 2^{0} \varphi} = e^{i 2 \pi \varphi} = e^{i 2 \ pi 0.\varphi_1 \varphi_2 ... \varphi_m}$ und

für $t=1$ wäre die Phase $e^{i 2 \pi 2^{1} \varphi} = e^{i 2 \pi \varphi_1} e^{i 2 \pi 0.\varphi_2 \ varphi_3 ... \varphi_m}$ und

für $t=2$, $e^{i 2 \pi 2^{2} \varphi} = e^{i 2 \pi 2 \varphi_1} e^{i 2 \pi \varphi_2} e^{i 2 \pi 0.\varphi_3 \varphi_4 ... \varphi_m}$ und

für $t=m-1$, $e^{i 2 \pi 2^{m-1} \varphi} = e^{i 2 \pi 2^{m-2} \varphi_1} e^{i 2 \pi 2^{m-3} \varphi_2} ... e^{i 2 \pi 2^{-1} \varphi_m} = e^{i 2 \pi 0.\varphi_m}$.

Beachten Sie, dass für den letzten Fall mit $t=m-1$ der Zustand von $q_0$ $|0\rangle + e^{i 2 \pi 0.\varphi_m}|1\rangle$ ist; $|+\rangle$ wenn $\varphi_m = 0$ und $|-\rangle$ wenn $\varphi_m = 1$ was zu den Ergebnissen $|0\rangle$ bzw. $|1\rangle$ führen würde, wenn es eingemessen wird x-Basis.

Im `the first step` des IPE-Algorithmus messen wir direkt das niedrigstwertige Bit der Phase $\varphi$, $\varphi_m$, indem wir die 2-Qubit-Register wie oben beschrieben initialisieren ( $q_0 \rightarrow |+\rangle$ and $q_1 \rightarrow |\Psi \rangle$ ), Ausführen von $2^{m-1}$ Kontroll-$U$-Operationen zwischen den Qubits und Messen von $q_0$ in der x-Basis.

Für `the second step` initialisieren wir die Systeme auf die gleiche Weise und wenden $2^{m-2}$ control-$U$ Operationen an. Die relative Phase in $q_0$ nach diesen Operationen ist jetzt $e^{i 2 \pi 0.\varphi_{m-1}\varphi_{m}}= e^{i 2 \pi 0.\varphi_{m- 1}} e^{i 2 \pi \varphi_m/4}$. Um das Phasenbit $\varphi_{m-1}$ zu extrahieren, führen Sie zunächst eine Phasenkorrektur durch, indem Sie um die $Z-$-Achse des Winkels $-2 \pi \varphi_m/4=-\pi \varphi_m/2$ drehen, was dazu führt, dass der Zustand von $q_0$ $|0\rangle + e^{i 2 \pi 0.\varphi_{m-1}} | ist 1 \rangle$. Führen Sie eine Messung an $q_0$ auf x-Basis durch, um das Phasenbit $\varphi_{m-1}$ zu erhalten.

Daher besteht der $k$te Schritt des IPE, der $\varphi_{m-k+1}$ erhält, aus der Registerinitialisierung ($q_0$ in $|+\rangle$, $q_1$ in $|\Psi \rangle$), die Anwendung von Control-$U$ $2^{mk}$ mal, eine Rotation um $Z$ mit Winkel $\omega_k = -2 \pi 0.0\varphi_{m-k+2} ... \varphi_m$ und eine Messung von $q_0$ in x-Basis: eine Hadamard-Transformation zu $q_0$ und eine Messung von $q_0$ in der Standardbasis. Beachten Sie, dass $q_1$ während des gesamten Algorithmus im Zustand $|\Psi\rangle$ bleibt.

<h3 style="font-size: 20px">3. Schätzen Sie die Phase des $T$-Gatters ab, das den IPE-Algorithmus implementiert.</h3>

Sehen Sie sich den `section 2. Example: T-gate` in [Kapitel 3.8 Quantum Phase Estimation](https://qiskit.org/textbook/ch-algorithms/quantum-phase-estimation.html) und den `section 4. Measuring in Different Bases` in [Kapitel 1.4 Single Qubit Gates](https://qiskit.org/textbook/ch-states/single-qubit-gates.html) an 

Wie wir bereits im Qiskit-Lehrbuch gelernt haben, wird die Phase eines T-Gatters genau mit drei Bits ausgedrückt. 

<h4 style="font-size: 17px">📓Schritt A. Ermitteln Sie das niedrigstwertige Phasenbit des $T$-Gatters, indem Sie die Schaltung <code>T_x3</code> richtig einrichten und den Wert der Variablen <code>x_3</code> zuweisen.</h4>

Im vorherigen Abschnitt erläutert `the first step` , wie die Schaltung aufgebaut wird, um das niederwertigste Phasenbit zu extrahieren. 

In [26]:
q = QuantumRegister(2)
c = ClassicalRegister(1)

T_x3 = QuantumCircuit(q,c)

########## your code goes here #######

##1 Initialization





##2 Apply control-U operator as many times as needed to get the least significant phase bit





##3 measure the anscillar qubit in x-basis





########## Simulate the circuit and assign the output value to the variable 'x_3' 
job = execute(T_x3, sim, shots=1, memory=True)
x_3 = int(job.result().get_memory()[0])

<h4 style="font-size: 17px">📓Schritt B. Extrahieren Sie das mittlere Phasenbit des $T$-Gatters, indem Sie die Schaltung <code>T_x2</code> mit Phasenkorrektur unter Verwendung des <code>x_3</code> -Werts aus Schritt A erstellen. Weisen Sie das Ergebnisbit der Variablen <code>x_2</code> zu.</h4>

Lesen Sie den `the second step` im vorherigen Abschnitt. 

In [27]:
q = QuantumRegister(2)
c = ClassicalRegister(1)

T_x2 = QuantumCircuit(q,c)

########### your code goes here ##########

##1 Initialization





##2 phase correction





##3 Apply control-U operator as many times as needed 





##4 measure the anscillar qubit in x-basis





######## Simulate the circuit and assign the output value to the variable 'x_2' 
job = execute(T_x2, sim, shots=1, memory=True)
x_2 = int(job.result().get_memory()[0])

<h4 style="font-size: 17px">📓Schritt C. Finden Sie das höchstwertige Phasenbit des $T$-Gatters und weisen Sie es der Variablen <code>x_1</code> zu.</h4>

In [28]:
q = QuantumRegister(2)
c = ClassicalRegister(1)

T_x1 = QuantumCircuit(q,c)

########### your code goes here #########

##1 Initialization





##2 phase correction





##3 Apply control-U operator as many times as needed to get the least significant phase bit





##4 measure the anscillar qubit in x-basis





########## Simulate the circuit and assign the output value to the variable 'x_1' 
job = execute(T_x1, sim, shots=1, memory=True)
x_1 = int(job.result().get_memory()[0])

Daher ist das $T$-Gate-Phasenbit, das Sie gefunden haben, 0.x_1x_2x_3. Führen Sie die folgende Zelle aus, um zu überprüfen, ob Ihre Antwort richtig ist, indem Sie Ihr Phasenbit `x_1x_2x_3` mit `001` vergleichen, der Antwort im Qiskit-Lehrbuch, die $\frac{1}{8}$ ( = `0.125` ), der Phase von $, entspricht T$-Tor.

In [5]:
T_phase_bits = '{}{}{}'.format(x_1, x_2, x_3) 
T_phase_bits == '001'

<h4 style="font-size: 17px">📓Schritt D. Konstruieren Sie die vollständige IPE-Schaltung und übergeben Sie sie an die Variable <code>qc_T</code> ; Setzen Sie alle Schritte, die Sie ausgeführt haben, in eine Schaltung, indem Sie bedingte Operationen verwenden und zurücksetzen.</h4>

Anstatt drei getrennte Schaltungen zu verwenden, um jeden Phasenbitwert zu extrahieren, bauen Sie eine Schaltung auf; Führen Sie eine Reset-Operation für das Hilfs-Qubit durch, nachdem jedes Bit in einem klassischen Register gemessen wurde. Daher benötigt die Schaltung für dieses Beispiel drei klassische Register; das niedrigstwertige Bit, gemessen in das klassische Register, c[0] und das höchstwertige Bit, gemessen in c[2]. Implementieren Sie einen Bedingungsoperator zwischen dem Hilfs-Qubit und dem klassischen Register für die Phasenkorrektur. 

In [6]:
##### your code goes here ######

















################
qc_T.draw()


<h4 style="font-size: 17px">Schritt E. Führen Sie die folgende Zelle aus, um die Simulation durchzuführen und das Ergebnis anzuzeigen.</h4>

In [7]:
count0 = execute(qc_T, sim).result().get_counts()
n=3
key_new = [str(int(key,2)/2**n) for key in list(count0.keys())]
count1 = dict(zip(key_new, count0.values()))

fig, ax = plt.subplots(1,2)
plot_histogram(count0, ax=ax[0])
plot_histogram(count1, ax=ax[1])
plt.tight_layout()

<h2 style="font-size:24px;">Teil 2: Vergleich zwischen QPE- und IPE-Ergebnissen bei Vorhandensein von Rauschen</h2>

<br>
<div style="background: #E8E7EB; border-radius: 5px;
-moz-border-radius: 5px;">
  <p style="background: #800080;
            border-radius: 5px 5px 0px 0px;
            padding: 10px 0px 10px 10px;
            font-size:18px;
            color:white;
            "><b>Tor</b></p>
    <p style=" padding: 0px 0px 10px 10px;
              font-size:16px;">Verstehen Sie, wie wichtig es ist, flache Schaltungsalgorithmen auf aktuellen lauten Quantencomputern zu implementieren.</p>
</div>

In Teil 2 von Lab 4 haben wir eine Quantum Phase Estimation (QPE)-Schaltung auf einem echten Quantengerät ausgeführt. Nachdem wir die Grenzen der Leistung aufgrund von Rauschen erkannt haben, das in aktuellen Quantensystemen auftritt, haben wir verschiedene Techniken angewendet, um seinen Einfluss auf das Ergebnis zu reduzieren. Das Endergebnis, das nach all diesen Verfahren erzielt wurde, ist jedoch noch lange nicht ideal. Hier implementieren wir den Iterative Phase Estimation (IPE)-Algorithmus, um den Effekt von Rauschen bei der Phasenschätzung weitgehend zu überwinden und das Ergebnis mit dem QPE-Ergebnis zu vergleichen. 

Um den Einfluss des Rauschens eines realen Quantensystems auf das Ergebnis zu untersuchen, werden wir verrauschte Simulationen einer IPE-Schaltung mit dem Qiskit Aer- `noise` durchführen, das ein vereinfachtes Rauschmodell für ein IBM-Quantensystem erzeugt. Um mehr über laute Simulationen zu erfahren, lesen Sie [hier](https://qiskit.org/documentation/tutorials/simulators/2_device_noise_simulation.html) .

Wie in Übung 4 überlegen wir, die Phase des `p` -Gatters mit $\theta = \frac{1}{3}$ abzuschätzen. Angenommen, die Genauigkeit der Schätzung, die wir hier wünschen, ist die gleiche wie wenn die QPE-Schaltung vier zählende Qubits hat, was die Anzahl der Iterationen und der klassischen Register bestimmt, die für die IPE-Schaltung erforderlich sind.

<h4 style="font-size: 17px">📓Schritt A. Wie viele klassische Register werden benötigt? Weisen Sie den Wert der Variablen <code>n</code> zu.</h4>

In [38]:
## your answer goes here
n = 

<h4 style="font-size: 17px">📓Schritt B. Konstruieren Sie die IPE-Schaltung in der folgenden Zelle.</h4>

In [8]:
q = QuantumRegister(2)
c = ClassicalRegister(n)

IPE = QuantumCircuit(q,c)

########## your code goes here ############















#####################
IPE.draw()

<h4 style="font-size: 17px">Schritt C. Führen Sie die Zelle unten aus, um die QPE-Schaltung für den Vergleich zu erstellen.</h4>

In [16]:
def qft(n):
    """Creates an n-qubit QFT circuit"""
    circuit = QuantumCircuit(n)
    def swap_registers(circuit, n):
        for qubit in range(n//2):
            circuit.swap(qubit, n-qubit-1)
        return circuit
    def qft_rotations(circuit, n):
        """Performs qft on the first n qubits in circuit (without swaps)"""
        if n == 0:
            return circuit
        n -= 1
        circuit.h(n)
        for qubit in range(n):
            circuit.cp(np.pi/2**(n-qubit), qubit, n)
        qft_rotations(circuit, n)
    
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit


# define the parameters
t, psi = 4, 1/3*np.pi*2

# building a circuit
QPE = QuantumCircuit(t+1,t)
QPE.h(range(t))
QPE.x(t)
for idx in range(t):
    QPE.cp(psi*2**idx, idx, t)
    
qft_dag = qft(t).to_gate().inverse()
qft_dag.label = 'QFT+'
QPE.append(qft_dag, range(t))
QPE.measure(range(t), range(t))

QPE.draw()

<h4 style="font-size: 17px">📓Schritt D. Transpilieren Sie die Schaltungen für das Backend <code>ibmq_Athens</code> .</h4>

Führen Sie die folgende Zelle aus, um die Eigenschaften des Backends `ibmq_Athens` zu überprüfen. Wählen Sie ein initial_layout aus und transpilieren Sie die IPE-Schaltungseinstellung `optimization_level` = 3 und speichern Sie die transpilierte Schaltung in der Variablen `IPE_trans` . Drucken Sie die Tiefe der transpilierten Schaltung aus.

In [60]:
from qiskit.test.mock import FakeAthens
import qiskit.tools.jupyter

backend = FakeAthens()
backend

In [10]:
######## your code to transpile IPE circuit goes here ########




#####################
print(IPE_trans.depth())

Führen Sie die Zelle unten aus, um die QPE-Schaltung zu transpilieren. 

In [11]:
num = 500
QPE_trans = transpile([QPE]*num, backend, optimization_level=3)
QPE_trans_depth = np.array([QPE_trans[idx].depth() for idx in range(num)])
print(min(QPE_trans_depth), max(QPE_trans_depth))
best_arg = np.argmin(QPE_trans_depth)
QPE_trans_best = QPE_trans[best_arg]

<h4 style="font-size: 17px">Schritt E. Führen Sie die folgenden Zellen aus, um die Rauschsimulation der transipilierten Schaltungen durchzuführen.</h4>

In [65]:
from qiskit.providers.aer.noise import NoiseModel

In [66]:
noise_model = NoiseModel.from_backend(backend)
shots = 20000

counts = execute([IPE_trans, QPE_trans_best], sim, noise_model=noise_model).result().get_counts()

<h4 style="font-size: 17px">Schritt F. Führen Sie die Zelle unten aus, um die exakten Ergebnisse der Phasenschätzung zu berechnen.</h4>

In [84]:
from qiskit.quantum_info import Statevector

QPE_exact = QuantumCircuit(t+1)
QPE_exact.h(range(t))
QPE_exact.x(t)
for idx in range(t):
    QPE_exact.cp(psi*2**idx, idx, t)
    
qft_dag = qft(t).to_gate().inverse()
qft_dag.label = 'QFT+'
QPE_exact.append(qft_dag, range(t))

#QPE_exact.draw('mpl')

state = Statevector.from_instruction(QPE_exact)
pmf = state.probabilities_dict(range(4))

<h4 style="font-size: 17px">Schritt G. Zeigen Sie die Vergleichsfigur, indem Sie die folgende Zelle ausführen.</h4>

In [12]:
def count_new(count):
    phi_est = np.array([round(int(key, 2)/2**t, 3) for key in list(count.keys())])
    key_new = list(map(str, phi_est))
    count_new = dict(zip(key_new, count.values()))
    return count_new

pmf_new = count_new(pmf)
count_IPE = count_new(counts[0])
count_QPE = count_new(counts[1])

fig, ax = plt.subplots(1, 2, figsize=(32,10))
fig.suptitle('QPE .vs. IPE for estimating $\\theta=1/3$', fontsize=23)
plot_histogram([pmf_new, count_QPE], ax=ax[0], legend=['No_noise', 'Athens_sim'])
plot_histogram([pmf_new, count_IPE], ax=ax[1], legend=['No_noise', 'Athens_sim'])
ax[0].set_title('QPE circuit result', fontsize=16)
ax[0].set_xlabel('$\\theta$', fontsize=16)
ax[1].set_title('IPE circuit result', fontsize=16)
ax[1].set_xlabel('$\\theta$', fontsize=16)
plt.show()

Wenn Sie die IPE-Schaltung erfolgreich erstellen, um die Phase abzuschätzen, $\theta = \frac{1}{3}$, erhalten Sie ähnliche Diagramme wie unten gezeigt. 

![](images/L6_QPEvsIPE.png)

<h4 style="font-size: 17px">📓Schritt G. Besprechen Sie die Ergebnisse.</h4>